In [4]:
import pandas as pd
import numpy as np

In [5]:
train = pd.read_csv('/kaggle/input/vserosai2024final/train.csv')
test = pd.read_csv('/kaggle/input/vserosai2024final/test.csv')
sample_submission = pd.read_csv('/kaggle/input/vserosai2024final/sample_submission.csv')
items = pd.read_parquet('/kaggle/input/vserosai2024final/items.parquet')

In [6]:
df_train = train.merge(items, left_on=['leftItemId'], right_on=['itemId']).merge(items, left_on=['rightItemId'], right_on=['itemId'], suffixes=('_left', '_right')).drop(['itemId_left', 'itemId_right','leftItemId','rightItemId'], axis=1)[['authorId_left','authorId_right','title_left','title_right','content_left','content_right']]

In [ ]:
!pip install transformers sentence_transformers -U

In [ ]:
!pip uninstall -y transformers 
!pip install transformers 

In [ ]:
!pip install sentence_transformers

In [1]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("cointegrated/rubert-tiny2",device='cuda')


2025-11-13 04:56:05.680565: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763009765.703317     198 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763009765.710243     198 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [2]:
from tqdm import tqdm
tqdm.pandas()


In [7]:
df_train['title_left_emb'] = df_train['title_left'].progress_apply(lambda x: model.encode(x))

100%|██████████| 211393/211393 [12:24<00:00, 283.85it/s]


In [8]:
df_train['title_right_emb'] = df_train['title_right'].progress_apply(lambda x: model.encode(x))


100%|██████████| 211393/211393 [12:18<00:00, 286.25it/s]


0.48302147

In [10]:

def cosine_similarity(vector1, vector2):
    dot_product = np.dot(vector1, vector2)
    norm_vector1 = np.linalg.norm(vector1)
    norm_vector2 = np.linalg.norm(vector2)
    
    if norm_vector1 == 0 or norm_vector2 == 0:
        return 0  
    
    return dot_product / (norm_vector1 * norm_vector2)


In [14]:
len(df_train)

211393

In [17]:
df_train['cos_sim'] = np.nan

In [47]:
df_test['cos_sim'] = np.nan


In [ ]:
df_train['cos_sim']

In [52]:
df_test.cos_sim.isna().sum()

0

In [24]:
import warnings
warnings.filterwarnings("ignore")


In [25]:
df_train['cos_sim'].loc[i] = cosine_similarity(df_train['title_left_emb'].loc[i], df_train['title_right_emb'].loc[i])


In [49]:
df_test

,authorId_left,authorId_right,title_left,title_right,content_left,content_right,title_left_emb,cos_sim
0,13964239620121035293,4720689653046454286,Боеприпас повышенного могущества «Молот»: для ...,Советский противотанковый ракетный комплекс со...,Проведение специальной военной операции (СВО)...,В семидесятых годах коломенское КБ Машинострое...,"[0.007172948, 0.010601718, -0.01127057, -0.105...",NaN
1,11120158660896222880,11120158660896222880,Как пленные солдаты Гитлера дурили своих надзи...,Фельдмаршал Клейст: как сидел в ГУЛАГе самый в...,"По мере того, как Красная армия одолевала ве...",Эвальд фон Клейст был в числе трех немецких ...,"[0.009333594, 0.00013587091, -0.08811437, -0.0...",NaN
2,15315573502962914082,15315573502962914082,"Как похудеть, составляя суточное меню по метод...",Суточное меню для похудения по методу ладони. ...,"Привет, привет 👋 всем!!! В прошлых двух стать...",Наконец-то я на практике опробовала метод ладо...,"[-0.014685041, -0.06297872, 0.02110693, -0.062...",NaN
3,15247908169280368273,6595135292879062524,В России будут строить новую гидроэлектростанц...,Индия вышла из российской программы истребител...,Уже неоднократно обсуждался проект расширения ...,Индия традиционно является серьезным партнеров...,"[0.03725082, -0.013889726, -0.008771135, -0.04...",NaN
4,339773650285143442,9525796211347359624,Как сегодня выглядели бы Александр Македонский...,"Как бы выглядел современный мир, населенный не...",Благодаря новейшим технологиям у современных л...,В далеком прошлом в мире существовало множеств...,"[0.03660106, -0.012185668, 0.024544839, -0.075...",NaN
...,...,...,...,...,...,...,...,...
51631,5670928200842894505,16953570589854824553,Почему в поселении Villas Las Estrellas можно ...,Как строят в других странах. Финляндия,В каждом населённом пункте есть собственные ...,Интересное с финской выставки домостроения Asu...,"[0.035348356, -0.008827483, -0.027921727, -0.0...",NaN
51632,14815274303115755876,1370761158478928838,"История о том, как Гердта и Гафта бандиты конь...",К 105-летию со дня рождения Зиновия Гердта: 8 ...,Об этом забавном случае рассказала актриса Анн...,"""С ним всегда все было пронзительно забавно. ...","[0.0070438082, -0.054757643, -0.04263749, -0.0...",NaN
51633,13078125282572797991,13078125282572797991,"5 главных ""нельзя"" для рассады баклажанов: от ...",Первая подкормка клубники весной: когда и чем ...,Баклажаны можно назвать весьма капризной и тре...,"Как ваши грядки, дорогие садоводы? Еще под суг...","[0.05582657, -0.015902953, -0.05041401, -0.073...",NaN
51634,14192093339635693828,14192093339635693828,"""То, что у меня такая красивая жена, делает мо...","""Цезарь покорил полмира, а Клеопатра покорила ...",Со второй женой Ливией. Источник фото: ego-zhe...,Источник фото: spletnik.ru Ок­руг­лый под­бо­р...,"[0.0070737344, -0.0011654634, -0.0099395625, -...",NaN


In [51]:
for i in tqdm(range(len(df_test)), total=len(df_test)):
    if df_test['cos_sim'].loc[i] ==np.nan:
        continue
    df_test['cos_sim'].loc[i] = cosine_similarity(df_test['title_left_emb'].loc[i], df_test['title_right_emb'].loc[i])


100%|██████████| 51636/51636 [00:17<00:00, 2953.47it/s]


In [ ]:
for i in tqdm(range(len(df_train)), total=len(df_train)):
    if df_train['cos_sim'].loc[i] ==np.nan:
        continue
    df_train['cos_sim'].loc[i] = cosine_similarity(df_train['title_left_emb'].loc[i], df_train['title_right_emb'].loc[i])


In [ ]:
df_train['cos_sim']=model.similarity(df_train['title_left_emb'], df_train['title_right_emb'])


In [28]:
df_train['cos_sim']

0         0.483021
1         0.452852
2         0.710431
3         0.602320
4         0.602879
            ...   
211388    0.561307
211389    0.602490
211390    0.751581
211391    0.661355
211392    0.476029
Name: cos_sim, Length: 211393, dtype: float64

In [32]:
X = df_train[['title_left', 'title_right','title_left_emb','title_right_emb','cos_sim']]

In [34]:
left_title_emb_array = np.vstack(df_train['title_left_emb'].values)
right_title_emb_array = np.vstack(df_train['title_right_emb'].values)


In [53]:
left_title_emb_array = np.vstack(df_test['title_left_emb'].values)
right_title_emb_array = np.vstack(df_test['title_right_emb'].values)


In [54]:
for i in range(left_title_emb_array.shape[1]):
    df_test[f'left_title_emb_{i}'] = left_title_emb_array[:, i]
    df_test[f'right_title_emb_{i}'] = right_title_emb_array[:, i]


In [55]:
df_test

,authorId_left,authorId_right,title_left,title_right,content_left,content_right,title_left_emb,cos_sim,title_right_emb,left_title_emb_0,...,left_title_emb_307,right_title_emb_307,left_title_emb_308,right_title_emb_308,left_title_emb_309,right_title_emb_309,left_title_emb_310,right_title_emb_310,left_title_emb_311,right_title_emb_311
0,13964239620121035293,4720689653046454286,Боеприпас повышенного могущества «Молот»: для ...,Советский противотанковый ракетный комплекс со...,Проведение специальной военной операции (СВО)...,В семидесятых годах коломенское КБ Машинострое...,"[0.007172948, 0.010601718, -0.01127057, -0.105...",0.578422,"[-0.03721971, -0.041738544, -0.010869676, -0.1...",0.007173,...,0.017898,0.000631,-0.085235,-0.033129,0.052834,0.072442,0.037164,0.068416,-0.074190,-0.108475
1,11120158660896222880,11120158660896222880,Как пленные солдаты Гитлера дурили своих надзи...,Фельдмаршал Клейст: как сидел в ГУЛАГе самый в...,"По мере того, как Красная армия одолевала ве...",Эвальд фон Клейст был в числе трех немецких ...,"[0.009333594, 0.00013587091, -0.08811437, -0.0...",0.689942,"[-0.01683787, -0.019146556, -0.06739467, -0.12...",0.009334,...,-0.010291,0.033980,-0.060419,0.026935,0.036258,0.006977,0.091058,0.047191,-0.110837,-0.064705
2,15315573502962914082,15315573502962914082,"Как похудеть, составляя суточное меню по метод...",Суточное меню для похудения по методу ладони. ...,"Привет, привет 👋 всем!!! В прошлых двух стать...",Наконец-то я на практике опробовала метод ладо...,"[-0.014685041, -0.06297872, 0.02110693, -0.062...",0.831872,"[-0.019136962, -0.008951636, 0.012875165, -0.0...",-0.014685,...,0.077994,0.020219,0.017736,0.008644,-0.007260,-0.060653,0.002651,-0.001301,-0.119500,-0.165993
3,15247908169280368273,6595135292879062524,В России будут строить новую гидроэлектростанц...,Индия вышла из российской программы истребител...,Уже неоднократно обсуждался проект расширения ...,Индия традиционно является серьезным партнеров...,"[0.03725082, -0.013889726, -0.008771135, -0.04...",0.550925,"[-0.041856874, -0.032162283, -0.0120348735, -0...",0.037251,...,-0.057971,-0.019887,-0.047056,0.039333,-0.030262,0.038700,0.088091,-0.033882,0.019850,-0.038516
4,339773650285143442,9525796211347359624,Как сегодня выглядели бы Александр Македонский...,"Как бы выглядел современный мир, населенный не...",Благодаря новейшим технологиям у современных л...,В далеком прошлом в мире существовало множеств...,"[0.03660106, -0.012185668, 0.024544839, -0.075...",0.661929,"[0.014301982, -0.0075968574, 0.08230522, -0.14...",0.036601,...,-0.048191,0.012981,-0.026262,-0.025377,-0.080618,-0.098958,0.016822,0.030807,-0.023860,0.030785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51631,5670928200842894505,16953570589854824553,Почему в поселении Villas Las Estrellas можно ...,Как строят в других странах. Финляндия,В каждом населённом пункте есть собственные ...,Интересное с финской выставки домостроения Asu...,"[0.035348356, -0.008827483, -0.027921727, -0.0...",0.459647,"[0.11811864, -0.052640166, 0.062287517, -0.054...",0.035348,...,-0.039530,0.001562,-0.026394,0.022523,-0.017292,0.040717,0.029695,0.052368,-0.016649,-0.038115
51632,14815274303115755876,1370761158478928838,"История о том, как Гердта и Гафта бандиты конь...",К 105-летию со дня рождения Зиновия Гердта: 8 ...,Об этом забавном случае рассказала актриса Анн...,"""С ним всегда все было пронзительно забавно. ...","[0.0070438082, -0.054757643, -0.04263749, -0.0...",0.634909,"[-0.022767473, -0.0077426736, -0.08101193, -0....",0.007044,...,-0.064028,-0.025988,-0.060787,-0.011460,-0.056203,0.005015,-0.003877,-0.001611,-0.005874,-0.042711
51633,13078125282572797991,13078125282572797991,"5 главных ""нельзя"" для рассады баклажанов: от ...",Первая подкормка клубники весной: когда и чем ...,Баклажаны можно назвать весьма капризной и тре...,"Как ваши грядки, дорогие садоводы? Еще под суг...","[0.05582657, -0.015902953, -0.05041401, -0.073...",0.580103,"[0.0635

In [36]:
X.drop(['title_left_emb', 'title_right_emb'],axis=1,inplace=True)

In [31]:
y = train['target'] 


In [39]:

# Разделение на train/validation
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [42]:


# Обучение CatBoost
from catboost import CatBoostClassifier

model_cb = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    loss_function='MultiClass',
    eval_metric='MultiClass',
    random_seed=42,
    verbose=100,
    text_features=['title_left', 'title_right']
    # cat_features=['left_authorId', 'right_authorId']
)

model_cb.fit(
    X_train, y_train,
    eval_set=(X_val, y_val),
    early_stopping_rounds=50,
    verbose=100
)

0:	learn: 1.6946288	test: 1.6935379	best: 1.6935379 (0)	total: 4.81s	remaining: 1h 20m 7s
100:	learn: 1.1332672	test: 1.1329865	best: 1.1329865 (100)	total: 7m 9s	remaining: 1h 3m 44s
200:	learn: 1.1109062	test: 1.1252467	best: 1.1252467 (200)	total: 14m 1s	remaining: 55m 45s
300:	learn: 1.0940406	test: 1.1224916	best: 1.1224818 (299)	total: 20m 43s	remaining: 48m 7s
400:	learn: 1.0800321	test: 1.1207006	best: 1.1207006 (400)	total: 27m 14s	remaining: 40m 40s
500:	learn: 1.0674314	test: 1.1194255	best: 1.1194255 (500)	total: 33m 36s	remaining: 33m 28s
600:	learn: 1.0553965	test: 1.1181652	best: 1.1181652 (600)	total: 40m 4s	remaining: 26m 36s
700:	learn: 1.0438483	test: 1.1172873	best: 1.1172838 (699)	total: 46m 29s	remaining: 19m 49s
800:	learn: 1.0328985	test: 1.1165033	best: 1.1164945 (799)	total: 52m 53s	remaining: 13m 8s
900:	learn: 1.0222579	test: 1.1159553	best: 1.1159553 (900)	total: 59m 12s	remaining: 6m 30s
999:	learn: 1.0118128	test: 1.1155852	best: 1.1155249 (986)	total: 1h

In [ ]:
y_pred = model_cb.predict(X_val)

In [64]:
from sklearn.metrics import f1_score, classification_report

# Предсказания на валидационной выборке
y_pred = model_cb.predict_proba(X_val)

In [72]:
(y_pred > 0.6)

AttributeError: 'numpy.ndarray' object has no attribute 'count'

6

In [43]:


# F1-score (macro - усредненный по всем классам)
f1_macro = f1_score(y_val, y_pred, average='macro')
print(f"F1-score (macro): {f1_macro:.4f}")

# F1-score (weighted - взвешенный по количеству примеров в каждом классе)
f1_weighted = f1_score(y_val, y_pred, average='weighted')
print(f"F1-score (weighted): {f1_weighted:.4f}")

# F1-score для каждого класса отдельно
f1_per_class = f1_score(y_val, y_pred, average=None)
print(f"F1-score по классам: {f1_per_class}")

# Полный отчет с precision, recall, f1 для каждого класса
print("\nClassification Report:")
print(classification_report(y_val, y_pred))

F1-score (macro): 0.4335
F1-score (weighted): 0.4848
F1-score по классам: [0.84159378 0.45281969 0.51202348 0.52065775 0.27366807 0.        ]

Classification Report:
                precision    recall  f1-score   support

       doubles       0.91      0.78      0.84       553
   no_relevant       0.53      0.40      0.45      8210
      relevant       0.50      0.53      0.51     14507
relevant_minus       0.46      0.59      0.52     14931
 relevant_plus       0.62      0.18      0.27      4065
   unavailable       0.00      0.00      0.00        13

      accuracy                           0.49     42279
     macro avg       0.50      0.41      0.43     42279
  weighted avg       0.51      0.49      0.48     42279



In [ ]:
print(f"Validation AUC: {model_cb.get_best_score()['validation']['AUC']:.4f}")


In [44]:
df_test = test.merge(items, left_on=['leftItemId'], right_on=['itemId']).merge(items, left_on=['rightItemId'], right_on=['itemId'], suffixes=('_left', '_right')).drop(['itemId_left', 'itemId_right','leftItemId','rightItemId'], axis=1)[['authorId_left','authorId_right','title_left','title_right','content_left','content_right']]


In [45]:
df_test['title_left_emb'] = df_test['title_left'].progress_apply(lambda x: model.encode(x))


100%|██████████| 51636/51636 [03:03<00:00, 281.86it/s]


In [50]:
df_test['title_right_emb'] = df_test['title_right'].progress_apply(lambda x: model.encode(x))


100%|██████████| 51636/51636 [02:59<00:00, 287.46it/s]


In [80]:
df_test[['title_left', 'title_right']]=df_test[['title_left', 'title_right']].astype(str)

In [82]:
df_test.isna().sum()

authorId_left          0
authorId_right         0
title_left             0
title_right            0
content_left           0
                      ..
right_title_emb_309    0
left_title_emb_310     0
right_title_emb_310    0
left_title_emb_311     0
right_title_emb_311    0
Length: 633, dtype: int64

In [85]:
ans = model_cb.predict(X_test)

In [87]:
submission = pd.read_csv('/kaggle/input/vserosai2024final/sample_submission.csv')

In [88]:
submission.head()

,Unnamed: 0,target
0,0,relevant_plus
1,1,relevant_minus
2,2,no_relevant
3,3,no_relevant
4,4,no_relevant


In [95]:
len(ans[:,0])

51636

In [96]:
len(submission)

51636

In [97]:
submission['target']=ans[:,0]

In [99]:
submission.to_csv('subm1.csv',index=False)

In [83]:
print(df_test.columns)
print(X_train.columns)

Index(['authorId_left', 'authorId_right', 'title_left', 'title_right',
       'content_left', 'content_right', 'title_left_emb', 'cos_sim',
       'title_right_emb', 'left_title_emb_0',
       ...
       'left_title_emb_307', 'right_title_emb_307', 'left_title_emb_308',
       'right_title_emb_308', 'left_title_emb_309', 'right_title_emb_309',
       'left_title_emb_310', 'right_title_emb_310', 'left_title_emb_311',
       'right_title_emb_311'],
      dtype='object', length=633)
Index(['title_left', 'title_right', 'cos_sim', 'left_title_emb_0',
       'right_title_emb_0', 'left_title_emb_1', 'right_title_emb_1',
       'left_title_emb_2', 'right_title_emb_2', 'left_title_emb_3',
       ...
       'left_title_emb_307', 'right_title_emb_307', 'left_title_emb_308',
       'right_title_emb_308', 'left_title_emb_309', 'right_title_emb_309',
       'left_title_emb_310', 'right_title_emb_310', 'left_title_emb_311',
       'right_title_emb_311'],
      dtype='object', length=627)


In [84]:
X_test=df_test[X_train.columns]